## Citibike Dataset



In [5]:
%matplotlib inline
import pandas as pd
import MySQLdb as mdb
import matplotlib 
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])
matplotlib.rcParams['figure.figsize'] = (20,10)

In [7]:
from sqlalchemy import create_engine

conn_string_read = 'mysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = 'localhost', 
    user = 'root',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4', db='citibike_new')
engine_read = create_engine(conn_string_read)



In [15]:
query = '''
SELECT station_id AS id, 
       AVG(num_bikes_available) AS available_bikes,
       AVG(num_docks_available) AS available_docks,
       AVG(num_bikes_disabled) AS disabled_bikes,
       AVG(num_docks_disabled) AS disabled_docks,
       AVG(num_bikes_available/(num_bikes_available+num_docks_available+num_bikes_disabled+num_docks_disabled)) AS percent_full, 
       DATE_FORMAT(last_reported, '%%Y-%%m-%%d %%H:00:00') AS communication_time 
FROM citibike_new.Status
WHERE  is_renting=1 AND is_installed=1 AND num_bikes_available+num_docks_available+num_bikes_disabled+num_docks_disabled>0
GROUP BY station_id, communication_time
'''
df = pd.read_sql(query, con=engine_read)

In [16]:
len(df)

880538

In [17]:
df.head(10)

,id,available_bikes,available_docks,disabled_bikes,disabled_docks,percent_full,communication_time
0,72,1.1333,34.0667,3.8,0.0,0.029040,2017-09-18 23:00:00
1,72,2.8000,32.2000,4.0,0.0,0.071787,2017-09-19 00:00:00
2,72,5.4444,29.5556,4.0,0.0,0.139589,2017-09-19 01:00:00
3,72,10.0000,25.0000,4.0,0.0,0.256425,2017-09-19 02:00:00
4,72,14.0000,21.0000,4.0,0.0,0.358980,2017-09-19 03:00:00
5,72,20.0000,15.0000,4.0,0.0,0.512800,2017-09-19 04:00:00
6,72,21.6250,13.3750,4.0,0.0,0.554500,2017-09-19 05:00:00
7,72,22.0000,13.0000,4.0,0.0,0.564100,2017-09-19 06:00:00
8,72,21.0000,14.0000,4.0,0.0,0.538500,2017-09-19 07:00:00
9,72,20.0000,15.0000,4.0,0.0,0.512820,2017-09-19 09:00:00


In [18]:
df['communication_time'] = pd.to_datetime(df['communication_time'], format='%Y-%m-%d %H:%M:%S')

In [26]:
df['total_docks'] =  df.available_bikes + df.available_docks + df.disabled_bikes + df.disabled_docks

In [27]:
df.dtypes

id                             int64
communication_time    datetime64[ns]
available_bikes              float64
available_docks              float64
disabled_bikes               float64
disabled_docks               float64
total_docks                  float64
percent_full                 float64
dtype: object

In [21]:
df.total_docks = df.total_docks.astype(int)

In [28]:
df = df[ ['id', 'communication_time', 'available_bikes', 'available_docks', 'disabled_bikes', 'disabled_docks', 'total_docks', 'percent_full' ] ]

In [29]:
df.sort_values(by = ['id', 'communication_time'], inplace=True)

In [30]:
df

,id,communication_time,available_bikes,available_docks,disabled_bikes,disabled_docks,total_docks,percent_full
0,72,2017-09-18 23:00:00,1.1333,34.0667,3.8000,0.0,39.0,0.029040
1,72,2017-09-19 00:00:00,2.8000,32.2000,4.0000,0.0,39.0,0.071787
2,72,2017-09-19 01:00:00,5.4444,29.5556,4.0000,0.0,39.0,0.139589
3,72,2017-09-19 02:00:00,10.0000,25.0000,4.0000,0.0,39.0,0.256425
4,72,2017-09-19 03:00:00,14.0000,21.0000,4.0000,0.0,39.0,0.358980
5,72,2017-09-19 04:00:00,20.0000,15.0000,4.0000,0.0,39.0,0.512800
6,72,2017-09-19 05:00:00,21.6250,13.3750,4.0000,0.0,39.0,0.554500
7,72,2017-09-19 06:00:00,22.0000,13.0000,4.0000,0.0,39.0,0.564100
8,72,2017-09-19 07:00:00,21.0000,14.0000,4.0000,0.0,39.0,0.538500
9,72,2017-09-19 09:00:00,20.0000,15.0000,4.0000,0.0,39.0,0.512820


### Writing a Pandas Dataframe in a MySQL Table

Now we will connect to our MySQL server. We will use the SQLAlchemy library of Python.

If you do not have the library, you need to install it by typing in the shell:

In [10]:
conn_string_write = 'mysql://{user}:{password}@{host}/?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org', 
    user = 'root',
    password = 'krjniJm6OuIyhEoo',
    encoding = 'utf8mb4')
engine_write = create_engine(conn_string_write)


Once we have connected successfully, we need to create our database:

In [12]:
# Query to create a database
db_name = 'citibike'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8mb4'".format(db=db_name)

# Create a database
engine_write.execute(create_db_query)
# And lets switch to the database
engine_write.execute("USE {db}".format(db=db_name))

In [29]:
table_name = 'stations'

df.to_sql(
    name = table_name, 
    schema = db_name, 
    con = engine_write,
    if_exists = 'replace',
    index = False, 
    chunksize = 1000)

In [ ]:
# Once we have the data in the table, we also specify a primary key
# If we had FOREIGN KEYS we can add them in the same way
# add_key_query = 'ALTER TABLE nypd ADD PRIMARY KEY(CMPLNT_NUM)'
# engine.execute(add_key_query)